# Layer 3 — Modeling & Evaluation

Train XGBoost pLTV model. Compute Lift, Precision@K, Recall@K,
Spearman, Calibration, ROC/AUC. Output `model_training.md` and `evaluation_metrics.md`.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from scipy.stats import spearmanr
from pathlib import Path
import sys

ROOT = Path('.').resolve().parent
sys.path.insert(0, str(ROOT))
from utils.reporting import write_report, md_table, save_plot, timestamp_line

DATA_PATH = ROOT / 'data' / 'cfm_pltv.csv'
if not DATA_PATH.exists():
    DATA_PATH = ROOT / 'data' / 'cfm_pltv_sample.csv'
df = pd.read_csv(DATA_PATH)
print(f'Loaded {len(df):,} rows')

In [ ]:
NUMERIC = [
    'login_rows_d7', 'active_days_d7', 'loginchannel_variety_d7',
    'network_variety_d7', 'clientversion_variety_d7', 'max_level_seen_d7',
    'max_ladderscore_d7', 'games_d7', 'win_rate_d7', 'avg_game_duration_d7',
    'avg_score_d7', 'kills_d7', 'deaths_d7', 'assists_d7', 'kd_d7',
    'max_level_game_d7', 'max_ladderlevel_d7', 'rev_d7', 'txn_cnt_d7',
]
CAT = ['media_source', 'first_country_code', 'first_os', 'first_login_channel']

feature_df = df[NUMERIC + CAT].copy()
for c in CAT:
    le = LabelEncoder()
    feature_df[c] = le.fit_transform(feature_df[c].astype(str))
feature_df['first_charge_day_offset_d7'] = df['first_charge_day_offset_d7'].fillna(-1)

X = feature_df
y = np.log1p(df['ltv30'].clip(lower=0))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Train: {len(X_train):,}  Test: {len(X_test):,}')

In [ ]:
# Train XGBoost
model = xgb.XGBRegressor(
    n_estimators=200, max_depth=6, learning_rate=0.05,
    objective='reg:squaredlogerror', random_state=42, n_jobs=-1,
)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

y_pred_log = model.predict(X_test)
y_true = np.expm1(y_test)
y_pred = np.expm1(y_pred_log)
print('Model trained.')

In [ ]:
# Feature importance
imp = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False).head(15)
fig_imp = px.bar(x=imp.values, y=imp.index, orientation='h',
                 title='Top 15 Feature Importances',
                 labels={'x': 'Importance', 'y': 'Feature'})
fig_imp.update_layout(yaxis=dict(autorange='reversed'))
fig_imp.show()

In [ ]:
# Evaluation metrics
rho, _ = spearmanr(y_true, y_pred)
print(f'Spearman ρ: {rho:.4f}')

# Lift curve
order = np.argsort(-y_pred.values)
sorted_actual = y_true.values[order]
cumrev = np.cumsum(sorted_actual) / sorted_actual.sum()
pcts = np.arange(1, len(cumrev) + 1) / len(cumrev)

sample_idx = np.linspace(0, len(pcts) - 1, 200, dtype=int)
fig_lift = go.Figure()
fig_lift.add_trace(go.Scatter(x=pcts[sample_idx]*100, y=cumrev[sample_idx]*100, name='Model'))
fig_lift.add_trace(go.Scatter(x=[0,100], y=[0,100], name='Random', line=dict(dash='dash')))
fig_lift.update_layout(title='Lift Curve', xaxis_title='% Users', yaxis_title='% Revenue')
fig_lift.show()

for k in [1, 5, 10]:
    idx = int(len(cumrev) * k / 100)
    print(f'Lift@{k}%: {cumrev[idx]:.1%}')

In [ ]:
# Precision@K / Recall@K
threshold = np.percentile(y_true, 90)
is_high = (y_true.values >= threshold).astype(int)
total_high = is_high.sum()

for k_pct in [1, 5, 10]:
    k = max(1, int(len(y_pred) * k_pct / 100))
    top_k_idx = order[:k]
    hits = is_high[top_k_idx].sum()
    prec = hits / k
    rec = hits / total_high if total_high > 0 else 0
    print(f'K={k_pct}%  Precision={prec:.3f}  Recall={rec:.3f}')

In [ ]:
# Calibration plot
n_bins = 10
bins = np.linspace(y_pred.min(), y_pred.max(), n_bins + 1)
bin_idx = np.digitize(y_pred.values, bins) - 1
bin_idx = np.clip(bin_idx, 0, n_bins - 1)

cal_pred, cal_actual = [], []
for b in range(n_bins):
    mask = bin_idx == b
    if mask.sum() > 0:
        cal_pred.append(y_pred.values[mask].mean())
        cal_actual.append(y_true.values[mask].mean())

fig_cal = go.Figure()
fig_cal.add_trace(go.Scatter(x=cal_pred, y=cal_actual, mode='markers+lines', name='Model'))
mx = max(max(cal_pred), max(cal_actual))
fig_cal.add_trace(go.Scatter(x=[0, mx], y=[0, mx], name='Perfect', line=dict(dash='dash')))
fig_cal.update_layout(title='Calibration Plot', xaxis_title='Predicted', yaxis_title='Actual')
fig_cal.show()

In [ ]:
# ROC / AUC for payer classification
is_payer = (y_true.values > 0).astype(int)
fpr, tpr, _ = roc_curve(is_payer, y_pred.values)
roc_auc = auc(fpr, tpr)
print(f'AUC: {roc_auc:.4f}')

fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, name=f'AUC={roc_auc:.3f}'))
fig_roc.add_trace(go.Scatter(x=[0,1], y=[0,1], name='Random', line=dict(dash='dash')))
fig_roc.update_layout(title='ROC Curve (Payer)', xaxis_title='FPR', yaxis_title='TPR')
fig_roc.show()